In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Define column names based on the UCI Machine Learning Repository description
column_names = [f'feature_{i}' for i in range(34)] + ['label']

file_path = "C:/Users/91959/Desktop/CODE/Robust-Logistic-Regression-with-Shift-Parameter-Estimation/Robust Logistic Regression [DATA DIRECTORY]/Linear Case/ionosphere/ionosphere.data"

df = pd.read_csv(file_path, header=None, names=column_names)

df.head(10)

In [ ]:
for i in df.columns:
    print(df[i].value_counts())
    print('-------------------------------')

In [ ]:
# Drop a single column
df = df.drop('feature_1', axis=1)  # axis=1 indicates columns

In [ ]:
df.shape

In [ ]:
df.info()

Data Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [ ]:
# Separate features and target variable
X = df.drop(columns=['label'])
y = df['label']

In [ ]:
# Standardize the feature set (zero mean, unit variance)
scaler = StandardScaler()
X = scaler.fit_transform(X)

def introduce_label_noise(y, noise_percentage=0.1):
    """
    Introduces label noise by flipping a percentage of majority class labels to the minority class.

    Args:
        y (pd.Series): The target variable.
        noise_percentage (float): The percentage of majority class labels to flip (e.g., 0.1 for 10%).

    Returns:
        pd.Series: The target variable with label noise.
    """

    value_counts = y.value_counts()
    majority_class = value_counts.idxmax()
    minority_class = value_counts.idxmin()

    print("Original class distribution:")
    print(value_counts)

    majority_indices = y[y == majority_class].index
    num_noise = int(len(majority_indices) * noise_percentage)

    noise_indices = np.random.choice(majority_indices, num_noise, replace=False)

    y_noisy = y.copy()
    y_noisy.loc[noise_indices] = minority_class

    print("\nClass distribution after introducing noise:")
    print(y_noisy.value_counts())

    return y_noisy

y = introduce_label_noise(y, noise_percentage=0.2) # x % noise

In [ ]:
# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Print dataset shapes
print(f"Training set size: {X_train.shape}, Test set size: {X_test.shape}")

In [ ]:
print("X_train type:",type(X_train))
print("X_test type:",type(X_test))
print("y_train type:",type(y_train))
print("y_test type:",type(y_test))

Implementation

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# ===========================
# SIGMOID FUNCTION
# ===========================
def sigmoid(z):
    """Compute the sigmoid function."""
    return 1 / (1 + np.exp(-z))

# ===========================
# THRESHOLDING FUNCTIONS FOR SHIFT PARAMETERS
# ===========================
def soft_threshold(u, lambda_, a):
    """Soft thresholding function for shift parameter estimation."""
    return a * np.minimum(u + lambda_, 0)

def hard_threshold(u, lambda_, a):
    """Hard thresholding function for shift parameter estimation."""
    return a * u * (u <= -lambda_)

In [ ]:
# ===========================
# ROBUST LOGISTIC REGRESSION WITH SHIFT PARAMETER ESTIMATION
# ===========================
def train_robust_logistic_regression(X, y, lr=0.01, epochs=1000, tol=1e-6, lambda_=1.0, a=2.0, threshold_type='soft'):
    """
    Train robust logistic regression using gradient descent with shift parameter estimation.
    """
    m, n = X.shape
    theta = np.zeros(n)
    gamma = np.zeros(m)
    prev_loss = float('inf')

    for epoch in range(epochs):
        # Step 1: Update weights theta using logistic regression with offset
        z = np.dot(X, theta)
        u = y * (z - gamma)
        h = sigmoid(z - gamma)
        gradient_theta = np.dot(X.T, (h - (y + 1) / 2)) / m
        theta -= lr * gradient_theta

        # Step 2: Update shift parameters gamma using thresholding
        z = np.dot(X, theta)
        u = y * z
        if threshold_type == 'soft':
            gamma = soft_threshold(u, lambda_, a)
        elif threshold_type == 'hard':
            gamma = hard_threshold(u, lambda_, a)
        else:
            raise ValueError("threshold_type must be 'soft' or 'hard'")

        # Step 3: Compute current loss with L1 penalty
        loss = np.mean(np.log(1 + np.exp(-u + gamma))) + lambda_ * np.sum(np.abs(gamma))

        # Early stopping condition
        if abs(prev_loss - loss) < tol:
            break
        prev_loss = loss

    return theta, gamma

# ===========================
# PREDICTION FUNCTION
# ===========================
def predict_robust_logistic_regression(X, theta):
    """
    Make predictions using trained robust logistic regression model.
    """
    probabilities = sigmoid(np.dot(X, theta))
    return np.where(probabilities >= 0.5, 1, -1)

In [ ]:
# ===========================
# APPLY TO YOUR DATASET
# ===========================

# Convert Pandas DataFrame to NumPy array
# X_train_np = X_train.values
# X_test_np = X_test.values

# Convert labels from {0,1} to {-1,1} while still in pandas Series form
y_train_np = 2 * y_train.values - 1  # Convert 0 → -1 and 1 → 1
y_test_np = 2 * y_test.values - 1    # Convert 0 → -1 and 1 → 1

In [ ]:
# Train robust logistic regression on your dataset
theta, gamma = train_robust_logistic_regression(
    X_train, y_train_np, lr=0.1, epochs=2000, tol=1e-6, lambda_=0.1, a=2.0, threshold_type="hard"
)

# Predict on test set
y_pred = predict_robust_logistic_regression(X_test, theta)

# Compute misclassification rate
misclassification_rate = 1 - accuracy_score(y_test_np, y_pred)

print(f"Test misclassification rate: {misclassification_rate:.4f}")

---

In [ ]:
# ===========================
# ROBUST LOGISTIC REGRESSION WITH ELASTIC NET
# ===========================
def train_robust_logistic_regression_elastic(X, y, lr=0.01, epochs=1000, tol=1e-6, lambda_=1.0, alpha=0.5, a=2.0, threshold_type='soft'):
    """
    Train robust logistic regression using gradient descent with shift parameter estimation and Elastic Net regularization.
    """
    m, n = X.shape
    theta = np.zeros(n)
    gamma = np.zeros(m)
    prev_loss = float('inf')

    for epoch in range(epochs):
        # Step 1: Update weights theta using logistic regression with offset
        z = np.dot(X, theta)
        u = y * (z - gamma)
        h = sigmoid(z - gamma)
        gradient_theta = np.dot(X.T, (h - (y + 1) / 2)) / m

        # Elastic Net penalty gradient:
        l1_grad = alpha * np.sign(theta)  # L1 (Lasso)
        l2_grad = (1 - alpha) * theta  # L2 (Ridge)

        # Apply Elastic Net regularization
        gradient_theta += lambda_ * (l1_grad + l2_grad)

        # Update weights
        theta -= lr * gradient_theta

        # Step 2: Update shift parameters gamma using thresholding
        z = np.dot(X, theta)
        u = y * z
        if threshold_type == 'soft':
            gamma = soft_threshold(u, lambda_, a)
        elif threshold_type == 'hard':
            gamma = hard_threshold(u, lambda_, a)
        else:
            raise ValueError("threshold_type must be 'soft' or 'hard'")

        # Step 3: Compute current loss with Elastic Net penalty
        l1_term = alpha * np.sum(np.abs(theta))
        l2_term = (1 - alpha) * np.sum(theta**2)
        loss = np.mean(np.log(1 + np.exp(-u + gamma))) + lambda_ * (l1_term + l2_term)

        # Early stopping condition
        if abs(prev_loss - loss) < tol:
            break
        prev_loss = loss

    return theta, gamma

In [ ]:
# ===========================
# FIT ON PREPROCESSED DATASET
# ===========================
def fit_and_evaluate_robust_logistic_regression(X_train, X_test, y_train, y_test, lambda_, alpha_elastic, a, threshold_type="soft"):
    """
    Fit the robust logistic regression model on the given dataset and evaluate its performance.
    """
    # # Ensure labels are in {-1, 1}
    # y_train = np.where(y_train == 1, 1, -1)
    # y_test = np.where(y_test == 1, 1, -1)

    # Train model
    theta, gamma = train_robust_logistic_regression_elastic(
        X_train, y_train, lr=0.1, epochs=2000, tol=1e-6,
        lambda_=lambda_, alpha=alpha_elastic, a=a, threshold_type=threshold_type
    )

    # Predict on test set
    y_pred = predict_robust_logistic_regression(X_test, theta)

    # Compute misclassification rate
    misclassification_rate = 1 - accuracy_score(y_test, y_pred)
    return misclassification_rate

In [ ]:
# ===========================
# USAGE EXAMPLE
# ===========================
# Assuming X_train, X_test, y_train, y_test are preprocessed and available
# Example usage (replace X_train, X_test, y_train, y_test with actual dataset variables)
misclassification_rate = fit_and_evaluate_robust_logistic_regression(X_train, X_test, y_train_np, y_test_np,
                                        lambda_=0.1, alpha_elastic=0.1, a=2.0, threshold_type="hard")
print(f"Test misclassification rate: {misclassification_rate:.4f}")

---

50 Iteration Experiment

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import statistics
import math

misclassification_rate_l1_lasso = []

# Define column names based on the UCI Machine Learning Repository description
column_names = [f'feature_{i}' for i in range(34)] + ['label']

file_path = "C:/Users/91959/Desktop/CODE/Robust-Logistic-Regression-with-Shift-Parameter-Estimation/Robust Logistic Regression [DATA DIRECTORY]/Linear Case/ionosphere/ionosphere.data"

df = pd.read_csv(file_path, header=None, names=column_names)

# Drop a single column
df = df.drop('feature_1', axis=1)  # axis=1 indicates columns

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Separate features and target variable
X = df.drop(columns=['label'])
y = df['label']

# Standardize the feature set (zero mean, unit variance)
scaler = StandardScaler()
X = scaler.fit_transform(X)

def introduce_label_noise(y, noise_percentage=0.1):
    """
    Introduces label noise by flipping a percentage of majority class labels to the minority class.

    Args:
        y (pd.Series): The target variable.
        noise_percentage (float): The percentage of majority class labels to flip (e.g., 0.1 for 10%).

    Returns:
        pd.Series: The target variable with label noise.
    """

    value_counts = y.value_counts()
    majority_class = value_counts.idxmax()
    minority_class = value_counts.idxmin()

    # print("Original class distribution:")
    # print(value_counts)

    majority_indices = y[y == majority_class].index
    num_noise = int(len(majority_indices) * noise_percentage)

    noise_indices = np.random.choice(majority_indices, num_noise, replace=False)

    y_noisy = y.copy()
    y_noisy.loc[noise_indices] = minority_class

    # print("\nClass distribution after introducing noise:")
    # print(y_noisy.value_counts())

    return y_noisy

y = introduce_label_noise(y, noise_percentage=0.1) # x % noise

# Methods for LogReg
# ===========================
# SIGMOID FUNCTION
# ===========================
def sigmoid(z):
    """Compute the sigmoid function."""
    return 1 / (1 + np.exp(-z))

# ===========================
# THRESHOLDING FUNCTIONS FOR SHIFT PARAMETERS
# ===========================
def soft_threshold(u, lambda_, a):
    """Soft thresholding function for shift parameter estimation."""
    return a * np.minimum(u + lambda_, 0)

def hard_threshold(u, lambda_, a):
    """Hard thresholding function for shift parameter estimation."""
    return a * u * (u <= -lambda_)

# ===========================
# ROBUST LOGISTIC REGRESSION WITH SHIFT PARAMETER ESTIMATION
# ===========================
def train_robust_logistic_regression(X, y, lr=0.01, epochs=1000, tol=1e-6, lambda_=1.0, a=2.0, threshold_type='soft'):
    """
    Train robust logistic regression using gradient descent with shift parameter estimation.
    """
    m, n = X.shape
    theta = np.zeros(n)
    gamma = np.zeros(m)
    prev_loss = float('inf')

    for epoch in range(epochs):
        # Step 1: Update weights theta using logistic regression with offset
        z = np.dot(X, theta)
        u = y * (z - gamma)
        h = sigmoid(z - gamma)
        gradient_theta = np.dot(X.T, (h - (y + 1) / 2)) / m
        theta -= lr * gradient_theta

        # Step 2: Update shift parameters gamma using thresholding
        z = np.dot(X, theta)
        u = y * z
        if threshold_type == 'soft':
            gamma = soft_threshold(u, lambda_, a)
        elif threshold_type == 'hard':
            gamma = hard_threshold(u, lambda_, a)
        else:
            raise ValueError("threshold_type must be 'soft' or 'hard'")

        # Step 3: Compute current loss with L1 penalty
        loss = np.mean(np.log(1 + np.exp(-u + gamma))) + lambda_ * np.sum(np.abs(gamma))

        # Early stopping condition
        if abs(prev_loss - loss) < tol:
            break
        prev_loss = loss

    return theta, gamma

# ===========================
# PREDICTION FUNCTION
# ===========================
def predict_robust_logistic_regression(X, theta):
    """
    Make predictions using trained robust logistic regression model.
    """
    probabilities = sigmoid(np.dot(X, theta))
    return np.where(probabilities >= 0.5, 1, -1)

def print_average_with_se(data_list):
    """
    Calculates and prints the average of a list with an error bar of ±SE.

    Args:
        data_list (list): The list of numerical data.
    """

    if not data_list:
        print("Error: Input list is empty.")
        return

    try:
        mean = statistics.mean(data_list)
        stdev = statistics.stdev(data_list)
        se = stdev / math.sqrt(len(data_list))

        print(f"Average: {mean:.4f} ± {se:.4f} SE")

    except statistics.StatisticsError:
        print("Error: Cannot calculate standard deviation. List must contain at least two elements.")
    except TypeError:
        print("Error: List elements must be numerical.")

for i in range(500):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=i)
    y_train_np = 2 * y_train.values - 1  # Convert 0 → -1 and 1 → 1
    y_test_np = 2 * y_test.values - 1    # Convert 0 → -1 and 1 → 1

    # Train robust logistic regression on your dataset
    theta, gamma = train_robust_logistic_regression(
        X_train, y_train_np, lr=0.1, epochs=2000, tol=1e-6, lambda_=0.1, a=2.0, threshold_type="hard"
    )

    # Predict on test set
    y_pred = predict_robust_logistic_regression(X_test, theta)

    # Compute misclassification rate
    misclassification_rate = 1 - accuracy_score(y_test_np, y_pred)
    misclassification_rate_l1_lasso.append(misclassification_rate)

print(misclassification_rate_l1_lasso)
print(f"Average misclassification rate over 50 runs L1 Lasso:")
print_average_with_se(misclassification_rate_l1_lasso)

[0.24528301886792447, 0.18867924528301883, 0.23584905660377353, 0.160377358490566, 0.23584905660377353, 0.18867924528301883, 0.18867924528301883, 0.24528301886792447, 0.160377358490566, 0.18867924528301883, 0.2264150943396226, 0.16981132075471694, 0.17924528301886788, 0.2547169811320755, 0.16981132075471694, 0.13207547169811318, 0.2264150943396226, 0.17924528301886788, 0.21698113207547165, 0.21698113207547165, 0.17924528301886788, 0.16981132075471694, 0.2075471698113207, 0.21698113207547165, 0.16981132075471694, 0.2075471698113207, 0.28301886792452835, 0.18867924528301883, 0.160377358490566, 0.13207547169811318, 0.2264150943396226, 0.2547169811320755, 0.23584905660377353, 0.17924528301886788, 0.2735849056603774, 0.2547169811320755, 0.26415094339622647, 0.2264150943396226, 0.18867924528301883, 0.18867924528301883, 0.2264150943396226, 0.2264150943396226, 0.2075471698113207, 0.23584905660377353, 0.16981132075471694, 0.23584905660377353, 0.2075471698113207, 0.2075471698113207, 0.2075471698

---

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import statistics
import math

misclassification_rate_elastic_net = []

# Define column names based on the UCI Machine Learning Repository description
column_names = [f'feature_{i}' for i in range(34)] + ['label']

file_path = "C:/Users/91959/Desktop/CODE/Robust-Logistic-Regression-with-Shift-Parameter-Estimation/Robust Logistic Regression [DATA DIRECTORY]/Linear Case/ionosphere/ionosphere.data"

df = pd.read_csv(file_path, header=None, names=column_names)

# Drop a single column
df = df.drop('feature_1', axis=1)  # axis=1 indicates columns

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Separate features and target variable
X = df.drop(columns=['label'])
y = df['label']

# Standardize the feature set (zero mean, unit variance)
scaler = StandardScaler()
X = scaler.fit_transform(X)

def introduce_label_noise(y, noise_percentage=0.1):
    """
    Introduces label noise by flipping a percentage of majority class labels to the minority class.

    Args:
        y (pd.Series): The target variable.
        noise_percentage (float): The percentage of majority class labels to flip (e.g., 0.1 for 10%).

    Returns:
        pd.Series: The target variable with label noise.
    """

    value_counts = y.value_counts()
    majority_class = value_counts.idxmax()
    minority_class = value_counts.idxmin()

    # print("Original class distribution:")
    # print(value_counts)

    majority_indices = y[y == majority_class].index
    num_noise = int(len(majority_indices) * noise_percentage)

    noise_indices = np.random.choice(majority_indices, num_noise, replace=False)

    y_noisy = y.copy()
    y_noisy.loc[noise_indices] = minority_class

    # print("\nClass distribution after introducing noise:")
    # print(y_noisy.value_counts())

    return y_noisy

y = introduce_label_noise(y, noise_percentage=0.1) # x % noise

# Methods for LogReg
# ===========================
# SIGMOID FUNCTION
# ===========================
def sigmoid(z):
    """Compute the sigmoid function."""
    return 1 / (1 + np.exp(-z))

# ===========================
# THRESHOLDING FUNCTIONS FOR SHIFT PARAMETERS
# ===========================
def soft_threshold(u, lambda_, a):
    """Soft thresholding function for shift parameter estimation."""
    return a * np.minimum(u + lambda_, 0)

def hard_threshold(u, lambda_, a):
    """Hard thresholding function for shift parameter estimation."""
    return a * u * (u <= -lambda_)

# ===========================
# ROBUST LOGISTIC REGRESSION WITH ELASTIC NET
# ===========================
def train_robust_logistic_regression_elastic(X, y, lr=0.01, epochs=1000, tol=1e-6, lambda_=1.0, alpha=0.5, a=2.0, threshold_type='soft'):
    """
    Train robust logistic regression using gradient descent with shift parameter estimation and Elastic Net regularization.
    """
    m, n = X.shape
    theta = np.zeros(n)
    gamma = np.zeros(m)
    prev_loss = float('inf')

    for epoch in range(epochs):
        # Step 1: Update weights theta using logistic regression with offset
        z = np.dot(X, theta)
        u = y * (z - gamma)
        h = sigmoid(z - gamma)
        gradient_theta = np.dot(X.T, (h - (y + 1) / 2)) / m

        # Elastic Net penalty gradient:
        l1_grad = alpha * np.sign(theta)  # L1 (Lasso)
        l2_grad = (1 - alpha) * theta  # L2 (Ridge)

        # Apply Elastic Net regularization
        gradient_theta += lambda_ * (l1_grad + l2_grad)

        # Update weights
        theta -= lr * gradient_theta

        # Step 2: Update shift parameters gamma using thresholding
        z = np.dot(X, theta)
        u = y * z
        if threshold_type == 'soft':
            gamma = soft_threshold(u, lambda_, a)
        elif threshold_type == 'hard':
            gamma = hard_threshold(u, lambda_, a)
        else:
            raise ValueError("threshold_type must be 'soft' or 'hard'")

        # Step 3: Compute current loss with Elastic Net penalty
        l1_term = alpha * np.sum(np.abs(theta))
        l2_term = (1 - alpha) * np.sum(theta**2)
        loss = np.mean(np.log(1 + np.exp(-u + gamma))) + lambda_ * (l1_term + l2_term)

        # Early stopping condition
        if abs(prev_loss - loss) < tol:
            break
        prev_loss = loss

    return theta, gamma

# ===========================
# PREDICTION FUNCTION
# ===========================
def predict_robust_logistic_regression(X, theta):
    """
    Make predictions using trained robust logistic regression model.
    """
    probabilities = sigmoid(np.dot(X, theta))
    return np.where(probabilities >= 0.5, 1, -1)

def print_average_with_se(data_list):
    """
    Calculates and prints the average of a list with an error bar of ±SE.

    Args:
        data_list (list): The list of numerical data.
    """

    if not data_list:
        print("Error: Input list is empty.")
        return

    try:
        mean = statistics.mean(data_list)
        stdev = statistics.stdev(data_list)
        se = stdev / math.sqrt(len(data_list))

        print(f"Average: {mean:.4f} ± {se:.4f} SE")

    except statistics.StatisticsError:
        print("Error: Cannot calculate standard deviation. List must contain at least two elements.")
    except TypeError:
        print("Error: List elements must be numerical.")

for i in range(500):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=i)
    y_train_np = 2 * y_train.values - 1  # Convert 0 → -1 and 1 → 1
    y_test_np = 2 * y_test.values - 1    # Convert 0 → -1 and 1 → 1

    # Train robust logistic regression on your dataset
    theta, gamma = train_robust_logistic_regression_elastic(
        X_train, y_train_np, lr=0.1, epochs=2000, tol=1e-6, lambda_=0.1, alpha=0.1 ,a=2.0,
        threshold_type="hard")

    # Predict on test set
    y_pred = predict_robust_logistic_regression(X_test, theta)

    # Compute misclassification rate
    misclassification_rate = 1 - accuracy_score(y_test_np, y_pred)
    misclassification_rate_elastic_net.append(misclassification_rate)

print(misclassification_rate_elastic_net)
print(f"Average misclassification rate over 50 runs Elastic Net:")
print_average_with_se(misclassification_rate_elastic_net)

[0.2075471698113207, 0.14150943396226412, 0.2264150943396226, 0.160377358490566, 0.16981132075471694, 0.15094339622641506, 0.18867924528301883, 0.17924528301886788, 0.14150943396226412, 0.160377358490566, 0.15094339622641506, 0.15094339622641506, 0.160377358490566, 0.15094339622641506, 0.17924528301886788, 0.16981132075471694, 0.18867924528301883, 0.16981132075471694, 0.24528301886792447, 0.1132075471698113, 0.21698113207547165, 0.24528301886792447, 0.14150943396226412, 0.17924528301886788, 0.2075471698113207, 0.1132075471698113, 0.21698113207547165, 0.14150943396226412, 0.160377358490566, 0.2075471698113207, 0.2075471698113207, 0.2264150943396226, 0.21698113207547165, 0.17924528301886788, 0.24528301886792447, 0.17924528301886788, 0.23584905660377353, 0.21698113207547165, 0.19811320754716977, 0.160377358490566, 0.19811320754716977, 0.23584905660377353, 0.17924528301886788, 0.2075471698113207, 0.17924528301886788, 0.19811320754716977, 0.13207547169811318, 0.16981132075471694, 0.20754716

---